In [ ]:
import numpy as np
import SimpleITK as sitk
import napari
from napari.utils import Colormap

def _clip_and_norm(arr, perc=(1,99)):
    lo, hi = np.percentile(arr, perc)
    arr = np.clip(arr, lo, hi)
    return (arr - lo) / (hi - lo) if hi > lo else np.zeros_like(arr)

def _get_channel(arr, n_comp, channel):
    """Return a 3D volume (Z,Y,X) from array that may be (Z,Y,X) or (Z,Y,X,C)."""
    if n_comp > 1:
        return arr[..., channel].astype(np.float32)
    return arr.astype(np.float32)

def _resample_to_reference(moving_img: sitk.Image, reference_img: sitk.Image) -> sitk.Image:
    rf = sitk.ResampleImageFilter()
    rf.SetReferenceImage(reference_img)
    rf.SetInterpolator(sitk.sitkLinear)
    return rf.Execute(moving_img)

def view_3d_two_files(
    nii_path_a: str,
    nii_path_b: str,
    channel_a: int = 0,
    channel_b: int = 0,
    color_a=(0,0,1),        # blue in 0..1
    color_b=(1,0,0),        # red in 0..1
    perc_a=(0.2, 99.8),
    perc_b=(0.2, 99.8),
    rendering='additive'    # 'attenuated_mip', 'mip', 'additive', 'translucent'
):
    imgA = sitk.ReadImage(nii_path_a)
    imgB = sitk.ReadImage(nii_path_b)

    if (imgA.GetSize() != imgB.GetSize()) or (imgA.GetSpacing() != imgB.GetSpacing()) \
       or (imgA.GetOrigin() != imgB.GetOrigin()) or (imgA.GetDirection() != imgB.GetDirection()):
        imgB = _resample_to_reference(imgB, imgA)

    arrA = sitk.GetArrayFromImage(imgA)  # (Z,Y,X) or (Z,Y,X,C)
    arrB = sitk.GetArrayFromImage(imgB)

    volA = _get_channel(arrA, imgA.GetNumberOfComponentsPerPixel(), channel_a)
    volB = _get_channel(arrB, imgB.GetNumberOfComponentsPerPixel(), channel_b)

    volA = _clip_and_norm(volA, perc=perc_a).astype(np.float32)
    volB = _clip_and_norm(volB, perc=perc_b).astype(np.float32)

    zyx_spacing = imgA.GetSpacing()[::-1]

    cmapA = Colormap(colors=[(0,0,0,0), (*color_a, 1.0)], name="const_blue")
    cmapB = Colormap(colors=[(0,0,0,0), (*color_b, 1.0)], name="const_red")

    v = napari.Viewer(ndisplay=3)
    v.add_image(
        volA, name="volA",
        colormap=("const_blue", cmapA),
        blending="translucent",
        rendering=rendering,
        contrast_limits=(0,1),
        opacity=1.0,
        scale=zyx_spacing
    )
    v.add_image(
        volB, name="volB",
        colormap=("const_red", cmapB),
        blending="translucent",
        rendering=rendering,
        contrast_limits=(0,1),
        opacity=1.0,
        scale=zyx_spacing
    )
    napari.run()

view_3d_two_files(
    nii_path_a=r"/Users/muhammadsohaib/Downloads/AT1_no_apotome_07_raw.nii.gz",   # blue
    nii_path_b=r"/Users/muhammadsohaib/Downloads/AT1_no_apotome_07_raw_sec.nii.gz", # red
    channel_a=0, channel_b=0,
    color_a=(0,0,1), color_b=(1,0,0),
    perc_a=(0.2,99.8), perc_b=(0.2,99.8),
    rendering='attenuated_mip'
)


In [ ]:
import numpy as np
import SimpleITK as sitk
import napari
from napari.utils import Colormap

def _clip_and_norm(arr, perc=(1,99)):
    lo, hi = np.percentile(arr, perc)
    arr = np.clip(arr, lo, hi)
    return (arr - lo) / (hi - lo) if hi > lo else np.zeros_like(arr)

def _get_channel(arr, n_comp, channel):
    """Return a 3D volume (Z,Y,X) from array that may be (Z,Y,X) or (Z,Y,X,C)."""
    if n_comp > 1:
        return arr[..., channel].astype(np.float32)
    return arr.astype(np.float32)

def _resample_to_reference(moving_img: sitk.Image, reference_img: sitk.Image) -> sitk.Image:
    """Resample moving to reference geometry (size, spacing, origin, direction)."""
    rf = sitk.ResampleImageFilter()
    rf.SetReferenceImage(reference_img)
    rf.SetInterpolator(sitk.sitkLinear)
    return rf.Execute(moving_img)

def view_3d_two_files(
    nii_path_a: str,
    nii_path_b: str,
    channel_a: int = 0,
    channel_b: int = 0,
    color_a=(0,0,1),        # blue in 0..1
    color_b=(1,0,0),        # red in 0..1
    perc_a=(0.2, 99.8),
    perc_b=(0.2, 99.8),
    rendering='additive'    # 'attenuated_mip', 'mip', 'additive', 'translucent'
):
    imgA = sitk.ReadImage(nii_path_a)
    imgB = sitk.ReadImage(nii_path_b)

    if (imgA.GetSize() != imgB.GetSize()) or (imgA.GetSpacing() != imgB.GetSpacing()) \
       or (imgA.GetOrigin() != imgB.GetOrigin()) or (imgA.GetDirection() != imgB.GetDirection()):
        imgB = _resample_to_reference(imgB, imgA)

    arrA = sitk.GetArrayFromImage(imgA)  # (Z,Y,X) or (Z,Y,X,C)
    arrB = sitk.GetArrayFromImage(imgB)

    volA = _get_channel(arrA, imgA.GetNumberOfComponentsPerPixel(), channel_a)
    volB = _get_channel(arrB, imgB.GetNumberOfComponentsPerPixel(), channel_b)

    volA = _clip_and_norm(volA, perc=perc_a).astype(np.float32)
    volB = _clip_and_norm(volB, perc=perc_b).astype(np.float32)

    zyx_spacing = imgA.GetSpacing()[::-1]

    cmapA = Colormap(colors=[(0,0,0,0), (*color_a, 1.0)], name="const_blue")
    cmapB = Colormap(colors=[(0,0,0,0), (*color_b, 1.0)], name="const_red")

    v = napari.Viewer(ndisplay=3)
    v.add_image(
        volA, name="volA",
        colormap=("const_blue", cmapA),
        blending="translucent",
        rendering=rendering,
        contrast_limits=(0,1),
        opacity=1.0,
        scale=zyx_spacing
    )
    v.add_image(
        volB, name="volB",
        colormap=("const_red", cmapB),
        blending="translucent",
        rendering=rendering,
        contrast_limits=(0,1),
        opacity=1.0,
        scale=zyx_spacing
    )
    napari.run()

view_3d_two_files(
    nii_path_a=r"/Users/muhammadsohaib/Downloads/Ca1D_IL11KO_HQ_apotome_05_processed_f.nii.gz",   # blue
    nii_path_b=r"/Users/muhammadsohaib/Downloads/Ca1D_IL11KO_HQ_apotome_05_processed_sec.nii.gz", # red
    channel_a=0, channel_b=0,
    color_a=(0,0,1), color_b=(1,0,0),
    perc_a=(0.2,99.8), perc_b=(0.2,99.8),
    rendering='attenuated_mip'
)
